# Task 1. Processing Data Stream (45%)
## Event Producer 3 (hospot_TERRA_streaming)

In [1]:
import csv
import pymongo
from pymongo import MongoClient
from datetime import datetime
from pprint import pprint

# Specify the host and port explicitly
client = MongoClient('192.168.100.7', 27017)

# Getting the database and collection
db = client["fit3182_asgn3"]
collection = db["hist_data"]

# Get the latest date of historic records
latest_date=''

pipeline = [
    {
        '$addFields': {
            'dateForSorting': {
                '$dateFromString': {'dateString': '$date', 'format': '%d/%m/%Y'}
            }
        }
    },
    {
        '$sort': {
            'dateForSorting': -1 # Sort in ascending order
        }
    },
    {
        '$limit': 1 
    }
]

cursor = collection.aggregate(pipeline)

for document in cursor:
    latest_date = document['dateForSorting']  # Store the value of the 'date' field in a variable
    print(latest_date)

2023-01-01 00:00:00


In [2]:
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import csv
import random
import datetime as dt

hostip = "192.168.100.7"  
csv_file = "hotspot_TERRA_streaming.csv"

def publish_message(producer_instance, topic_name, key, value):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(value, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        producer_instance.flush()
        print('Message published successfully. Data: ' + str(value))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))

def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[f'{hostip}:9092'],
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting to Kafka.')
        print(str(ex))
    finally:
        return _producer

    
if __name__ == '__main__':
    topic = 'Climate_Hotspot_Stream'
    
    print('Publishing records..')
    producer = connect_kafka_producer()

    # Read from hotspot_TERRA_streaming.csv
    with open(csv_file, 'r') as file:
        reader = csv.DictReader(file)
        
        # Add one day to the latest date
        latest_date += dt.timedelta(days=1)

        # Add 2 fields ('producer', 'date') to each climate record
        for row in reader:
            row['producer'] = 'TERRA'
            row['datetime'] = latest_date.strftime("%Y-%m-%dT%H:%M:%S")
            data = dumps(row)
            publish_message(producer, topic, 'parsed', data)
            
            # Increment the datetime by 1/5 day in every 2 seconds
            sleep(2)
            latest_date += dt.timedelta(days=1/5) 

Publishing records..
Message published successfully. Data: {"latitude": "-37.966", "longitude": "145.051", "confidence": "78", "surface_temperature_celcius": "68", "producer": "TERRA", "datetime": "2023-01-02T00:00:00"}
Message published successfully. Data: {"latitude": "-35.541", "longitude": "143.311", "confidence": "82", "surface_temperature_celcius": "63", "producer": "TERRA", "datetime": "2023-01-02T04:48:00"}
Message published successfully. Data: {"latitude": "-35.554", "longitude": "143.307", "confidence": "67", "surface_temperature_celcius": "53", "producer": "TERRA", "datetime": "2023-01-02T09:36:00"}
Message published successfully. Data: {"latitude": "-35.543", "longitude": "143.316", "confidence": "86", "surface_temperature_celcius": "67", "producer": "TERRA", "datetime": "2023-01-02T14:24:00"}
Message published successfully. Data: {"latitude": "-37.708", "longitude": "145.1", "confidence": "80", "surface_temperature_celcius": "54", "producer": "TERRA", "datetime": "2023-01-

Message published successfully. Data: {"latitude": "-37.59", "longitude": "149.31", "confidence": "51", "surface_temperature_celcius": "42", "producer": "TERRA", "datetime": "2023-01-10T09:36:00"}
Message published successfully. Data: {"latitude": "-37.633", "longitude": "149.264", "confidence": "64", "surface_temperature_celcius": "42", "producer": "TERRA", "datetime": "2023-01-10T14:24:00"}
Message published successfully. Data: {"latitude": "-37.585", "longitude": "149.336", "confidence": "100", "surface_temperature_celcius": "54", "producer": "TERRA", "datetime": "2023-01-10T19:12:00"}
Message published successfully. Data: {"latitude": "-37.58", "longitude": "149.331", "confidence": "69", "surface_temperature_celcius": "35", "producer": "TERRA", "datetime": "2023-01-11T00:00:00"}
Message published successfully. Data: {"latitude": "-37.583", "longitude": "149.316", "confidence": "100", "surface_temperature_celcius": "49", "producer": "TERRA", "datetime": "2023-01-11T04:48:00"}
Messag

Message published successfully. Data: {"latitude": "-34.965", "longitude": "142.031", "confidence": "70", "surface_temperature_celcius": "55", "producer": "TERRA", "datetime": "2023-01-18T19:12:00"}
Message published successfully. Data: {"latitude": "-37.978", "longitude": "145.623", "confidence": "84", "surface_temperature_celcius": "59", "producer": "TERRA", "datetime": "2023-01-19T00:00:00"}
Message published successfully. Data: {"latitude": "-37.976", "longitude": "145.649", "confidence": "88", "surface_temperature_celcius": "64", "producer": "TERRA", "datetime": "2023-01-19T04:48:00"}
Message published successfully. Data: {"latitude": "-37.758", "longitude": "144.693", "confidence": "85", "surface_temperature_celcius": "59", "producer": "TERRA", "datetime": "2023-01-19T09:36:00"}
Message published successfully. Data: {"latitude": "-37.862", "longitude": "144.175", "confidence": "87", "surface_temperature_celcius": "65", "producer": "TERRA", "datetime": "2023-01-19T14:24:00"}
Messa

Message published successfully. Data: {"latitude": "-36.765", "longitude": "144.176", "confidence": "74", "surface_temperature_celcius": "47", "producer": "TERRA", "datetime": "2023-01-27T04:48:00"}
Message published successfully. Data: {"latitude": "-35.325", "longitude": "143.497", "confidence": "77", "surface_temperature_celcius": "36", "producer": "TERRA", "datetime": "2023-01-27T09:36:00"}
Message published successfully. Data: {"latitude": "-35.321", "longitude": "143.502", "confidence": "100", "surface_temperature_celcius": "48", "producer": "TERRA", "datetime": "2023-01-27T14:24:00"}
Message published successfully. Data: {"latitude": "-35.323", "longitude": "143.509", "confidence": "100", "surface_temperature_celcius": "53", "producer": "TERRA", "datetime": "2023-01-27T19:12:00"}
Message published successfully. Data: {"latitude": "-36.156", "longitude": "143.568", "confidence": "67", "surface_temperature_celcius": "42", "producer": "TERRA", "datetime": "2023-01-28T00:00:00"}
Mes

Message published successfully. Data: {"latitude": "-37.461", "longitude": "148.109", "confidence": "69", "surface_temperature_celcius": "33", "producer": "TERRA", "datetime": "2023-02-04T14:24:00"}
Message published successfully. Data: {"latitude": "-37.459", "longitude": "148.092", "confidence": "92", "surface_temperature_celcius": "42", "producer": "TERRA", "datetime": "2023-02-04T19:12:00"}
Message published successfully. Data: {"latitude": "-37.236", "longitude": "141.176", "confidence": "68", "surface_temperature_celcius": "37", "producer": "TERRA", "datetime": "2023-02-05T00:00:00"}
Message published successfully. Data: {"latitude": "-37.227", "longitude": "141.151", "confidence": "59", "surface_temperature_celcius": "35", "producer": "TERRA", "datetime": "2023-02-05T04:48:00"}
Message published successfully. Data: {"latitude": "-37.242", "longitude": "141.153", "confidence": "77", "surface_temperature_celcius": "43", "producer": "TERRA", "datetime": "2023-02-05T09:36:00"}
Messa

Message published successfully. Data: {"latitude": "-37.087", "longitude": "145.37", "confidence": "64", "surface_temperature_celcius": "33", "producer": "TERRA", "datetime": "2023-02-13T00:00:00"}
Message published successfully. Data: {"latitude": "-37.758", "longitude": "148.721", "confidence": "76", "surface_temperature_celcius": "35", "producer": "TERRA", "datetime": "2023-02-13T04:48:00"}
Message published successfully. Data: {"latitude": "-37.561", "longitude": "148.032", "confidence": "73", "surface_temperature_celcius": "34", "producer": "TERRA", "datetime": "2023-02-13T09:36:00"}
Message published successfully. Data: {"latitude": "-37.446", "longitude": "148.102", "confidence": "100", "surface_temperature_celcius": "65", "producer": "TERRA", "datetime": "2023-02-13T14:24:00"}
Message published successfully. Data: {"latitude": "-37.448", "longitude": "148.114", "confidence": "72", "surface_temperature_celcius": "59", "producer": "TERRA", "datetime": "2023-02-13T19:12:00"}
Messa

Message published successfully. Data: {"latitude": "-36.984", "longitude": "148.25", "confidence": "50", "surface_temperature_celcius": "35", "producer": "TERRA", "datetime": "2023-02-21T09:36:00"}
Message published successfully. Data: {"latitude": "-37.335", "longitude": "148.064", "confidence": "88", "surface_temperature_celcius": "60", "producer": "TERRA", "datetime": "2023-02-21T14:24:00"}
Message published successfully. Data: {"latitude": "-37.347", "longitude": "148.019", "confidence": "61", "surface_temperature_celcius": "36", "producer": "TERRA", "datetime": "2023-02-21T19:12:00"}
Message published successfully. Data: {"latitude": "-37.569", "longitude": "148.024", "confidence": "87", "surface_temperature_celcius": "60", "producer": "TERRA", "datetime": "2023-02-22T00:00:00"}
Message published successfully. Data: {"latitude": "-37.585", "longitude": "148.032", "confidence": "80", "surface_temperature_celcius": "50", "producer": "TERRA", "datetime": "2023-02-22T04:48:00"}
Messag

Message published successfully. Data: {"latitude": "-38.3982", "longitude": "147.0509", "confidence": "78", "surface_temperature_celcius": "51", "producer": "TERRA", "datetime": "2023-03-01T14:24:00"}
Message published successfully. Data: {"latitude": "-37.8641", "longitude": "143.4276", "confidence": "100", "surface_temperature_celcius": "51", "producer": "TERRA", "datetime": "2023-03-01T19:12:00"}
Message published successfully. Data: {"latitude": "-37.8658", "longitude": "143.4162", "confidence": "92", "surface_temperature_celcius": "43", "producer": "TERRA", "datetime": "2023-03-02T00:00:00"}
Message published successfully. Data: {"latitude": "-37.8475", "longitude": "147.2512", "confidence": "76", "surface_temperature_celcius": "50", "producer": "TERRA", "datetime": "2023-03-02T04:48:00"}
Message published successfully. Data: {"latitude": "-37.8565", "longitude": "147.2532", "confidence": "71", "surface_temperature_celcius": "46", "producer": "TERRA", "datetime": "2023-03-02T09:36

Message published successfully. Data: {"latitude": "-36.2669", "longitude": "143.1906", "confidence": "61", "surface_temperature_celcius": "45", "producer": "TERRA", "datetime": "2023-03-09T19:12:00"}
Message published successfully. Data: {"latitude": "-36.2693", "longitude": "143.1768", "confidence": "94", "surface_temperature_celcius": "85", "producer": "TERRA", "datetime": "2023-03-10T00:00:00"}
Message published successfully. Data: {"latitude": "-36.3769", "longitude": "144.7505", "confidence": "55", "surface_temperature_celcius": "39", "producer": "TERRA", "datetime": "2023-03-10T04:48:00"}
Message published successfully. Data: {"latitude": "-36.059", "longitude": "143.7718", "confidence": "69", "surface_temperature_celcius": "44", "producer": "TERRA", "datetime": "2023-03-10T09:36:00"}
Message published successfully. Data: {"latitude": "-36.0201", "longitude": "143.7775", "confidence": "62", "surface_temperature_celcius": "41", "producer": "TERRA", "datetime": "2023-03-10T14:24:0

KeyboardInterrupt: 